In [41]:
import pandas as pd
import re

from tqdm.auto import tqdm
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
from actions import (
    remove_typos,
    CalculateWeightAndTrimAction,
    CheckinMsgProcessor,
    CreateLoadingInstructionAction,
    CreateLoadsheetAction,
    CreateZFWMessageAction,
    EstimateStorePaxDataAction,
    RampFinalAction,
    SendFuelOrderAction,
    SendLoadingInstructionAction,
    SendLoadsheetAction,
    SetActualBagWeightIndicatorAction,
    SetCKIPaxDistributionAction,
    StoreAircraftDataAction,
    StorePaxDataAction,
    StorePaxDataGuiAction,
    StoreRegistrationAndConfigurationAc,
    TdmCreateLoadingInstructionAction,
    TransferCargoAction,
    TransferCheckinDataAction,
    UpdateEstimatesAction,
    UpdateFuelDataAction,
    UpdateLoadTableAction,
    UpdateTransitLoadTableAction,
)



In [3]:
# Define the file paths
parquet_file_abcd = Path("../data/ABCD_tripfiles.parquet")
parquet_file_zyxw = Path("../data/ZYXW_tripfiles.parquet")
parquet_file_mnop = Path("../data/MNOP_tripfiles.parquet")

parquet_file_abcd_conv = Path("../data/ABCD_tripfiles_conv.parquet")
parquet_file_mnop_conv = Path("../data/MNOP_tripfiles_conv.parquet")
parquet_file_zyxw_conv = Path("../data/ZYXW_tripfiles_conv.parquet")

parquet_file_abcd_weights = Path("../data/ABCD_tripfiles_weights.parquet")
parquet_file_mnop_weights = Path("../data/MNOP_tripfiles_weights.parquet")
parquet_file_zyxw_weights = Path("../data/ZYXW_tripfiles_weights.parquet")

NameError: name 'Path' is not defined

In [2]:
df_abcd = pd.read_parquet(parquet_file_abcd)
df_mnop = pd.read_parquet(parquet_file_mnop)
df_zyxw = pd.read_parquet(parquet_file_zyxw)

NameError: name 'pd' is not defined

In [44]:
# print(round(df_abcd.memory_usage(deep=True).sum() / 1024**2, 2), "MB")
# print(round(df_mnop.memory_usage(deep=True).sum() / 1024**2, 2), "MB")
# print(round(df_zyxw.memory_usage(deep=True).sum() / 1024**2, 2), "MB")

In [45]:
import importlib

importlib.reload(remove_typos)
importlib.reload(CalculateWeightAndTrimAction)
importlib.reload(CheckinMsgProcessor)
importlib.reload(CreateLoadingInstructionAction)
importlib.reload(CreateLoadsheetAction)
importlib.reload(CreateZFWMessageAction)
importlib.reload(EstimateStorePaxDataAction)
importlib.reload(RampFinalAction)
importlib.reload(SendFuelOrderAction)
importlib.reload(SendLoadingInstructionAction)
importlib.reload(SendLoadsheetAction)
importlib.reload(SetActualBagWeightIndicatorAction)
importlib.reload(SetCKIPaxDistributionAction)
importlib.reload(StoreAircraftDataAction)
importlib.reload(StorePaxDataAction)
importlib.reload(StorePaxDataGuiAction)
importlib.reload(StoreRegistrationAndConfigurationAc)
importlib.reload(TdmCreateLoadingInstructionAction)
importlib.reload(TransferCargoAction)
importlib.reload(TransferCheckinDataAction)
importlib.reload(UpdateEstimatesAction)
importlib.reload(UpdateFuelDataAction)
importlib.reload(UpdateLoadTableAction)
importlib.reload(UpdateTransitLoadTableAction)

action_extractors = {
    "CalculateWeightAndTrimAction": CalculateWeightAndTrimAction.extract,
    "CheckinMsgProcessor": CheckinMsgProcessor.extract,
    "CreateLoadingInstructionAction": CreateLoadingInstructionAction.extract,
    "CreateLoadsheetAction": CreateLoadsheetAction.extract,
    "CreateZFWMessageAction": CreateZFWMessageAction.extract,
    "EstimateStorePaxDataAction": EstimateStorePaxDataAction.extract,
    "RampFinalAction": RampFinalAction.extract,
    "SendFuelOrderAction": SendFuelOrderAction.extract,
    "SendLoadingInstructionAction": SendLoadingInstructionAction.extract,
    "SendLoadsheetAction": SendLoadsheetAction.extract,
    "SetActualBagWeightIndicatorAction": SetActualBagWeightIndicatorAction.extract,
    "SetCKIPaxDistributionAction": SetCKIPaxDistributionAction.extract,
    "StoreAircraftDataAction": StoreAircraftDataAction.extract,
    "StorePaxDataAction": StorePaxDataAction.extract,
    "StorePaxDataGuiAction": StorePaxDataGuiAction.extract,
    "StoreRegistrationAndConfigurationAc": StoreRegistrationAndConfigurationAc.extract,
    "TdmCreateLoadingInstructionAction": TdmCreateLoadingInstructionAction.extract,
    "TransferCargoAction": TransferCargoAction.extract,
    "TransferCheckinDataAction": TransferCheckinDataAction.extract,
    "UpdateEstimatesAction": UpdateEstimatesAction.extract,
    "UpdateFuelDataAction": UpdateFuelDataAction.extract,
    "UpdateLoadTableAction": UpdateLoadTableAction.extract,
    "UpdateTransitLoadTableAction": UpdateTransitLoadTableAction.extract,
}

In [46]:
def extract_df(
    df: pd.DataFrame, progress_bar: bool = False, label: str | None = None
) -> pd.DataFrame:
    """Extract specific data based on predefined action extractors, optionally displaying a progress bar and labels.

    This function iterates over a dictionary of action names and their associated extractor functions, applying each
    extractor to the relevant entries in the DataFrame. The results are stored in new columns in the DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame from which data will be extracted.
            It must contain columns that match the keys in the action_extractors dictionary.
        progress_bar (bool, optional): If True, displays a progress bar during the data extraction process.
            Useful for visual feedback during long operations. Defaults to False.
        label (str | None, optional): An optional label that prefixes the print statements for better traceability during debugging.
            If None, only the action name is printed. Defaults to None.

    Returns:
        pd.DataFrame: The original DataFrame with additional columns containing the extracted data.
    """

    if progress_bar:
        tqdm.pandas()
    for action_name, extractor in action_extractors.items():
        if extractor is not None:
            if len(df[df.action_name == action_name]) == 0:
                print(label, action_name, "not found in DataFrame")
                continue

            if label:
                print(label, action_name)
            else:
                print(action_name)
            if progress_bar:
                df[f"data_{action_name}"] = df[df.action_name == action_name][
                    "entry_details"
                ].progress_apply(extractor)
            else:
                df[f"data_{action_name}"] = df[df.action_name == action_name][
                    "entry_details"
                ].apply(extractor)

    return df

In [47]:
def process_df(df, file_path, label, progress_bar=False):

    df_conv = extract_df(
        df,
        progress_bar=progress_bar,
        label=label,
    )
    df_conv.to_parquet(file_path, engine="pyarrow", compression="brotli")

In [48]:

# Create a ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = [
        executor.submit(process_df, df_abcd, parquet_file_abcd_conv, "ABCD"),
        executor.submit(process_df, df_mnop, parquet_file_mnop_conv, "MNOP"),
        executor.submit(process_df, df_zyxw, parquet_file_zyxw_conv, "ZYXW"),
    ]

    # Optional: Wait for all futures to complete
    for future in futures:
        future.result()  # This will re-raise any exceptions that occurred during task execution

ZYXW CalculateWeightAndTrimAction
ABCD CalculateWeightAndTrimAction
MNOP CalculateWeightAndTrimAction
ZYXW CheckinMsgProcessor
ZYXW CreateLoadingInstructionAction
ZYXW CreateLoadsheetAction
ZYXW CreateZFWMessageAction
ZYXW EstimateStorePaxDataAction
ZYXW RampFinalAction
ZYXW SendFuelOrderAction
ZYXW SendLoadingInstructionAction
ABCD CheckinMsgProcessor
ZYXW SendLoadsheetAction
ZYXW SetActualBagWeightIndicatorAction
ZYXW SetCKIPaxDistributionAction
ZYXW StoreAircraftDataAction
ABCD CreateLoadingInstructionAction
ZYXW StorePaxDataAction
ABCD CreateLoadsheetAction
ABCD CreateZFWMessageAction
ABCD EstimateStorePaxDataAction
ZYXW StorePaxDataGuiAction
ABCD RampFinalAction
ZYXW StoreRegistrationAndConfigurationAc
ABCD SendFuelOrderAction not found in DataFrame
ABCD SendLoadingInstructionAction
ABCD SendLoadsheetAction
ABCD SetActualBagWeightIndicatorAction
ZYXW TdmCreateLoadingInstructionAction
ABCD SetCKIPaxDistributionAction not found in DataFrame
ABCD StoreAircraftDataAction
ZYXW Transfer

In [ ]:
def match_df_to_actions(
    df: pd.DataFrame, progress_bar: bool = False, label: str | None = None
) -> pd.DataFrame:
    """Extract specific data based on predefined action extractors, optionally displaying a progress bar and labels.

    This function iterates over a dictionary of action names and their associated extractor functions, applying each
    extractor to the relevant entries in the DataFrame. The results are stored in new columns in the DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame from which data will be extracted.
            It must contain columns that match the keys in the action_extractors dictionary.
        progress_bar (bool, optional): If True, displays a progress bar during the data extraction process.
            Useful for visual feedback during long operations. Defaults to False.
        label (str | None, optional): An optional label that prefixes the print statements for better traceability during debugging.
            If None, only the action name is printed. Defaults to None.

    Returns:
        pd.DataFrame: The original DataFrame with additional columns containing the extracted data.
    """

    if progress_bar:
        tqdm.pandas()
    for action_name, extractor in action_extractors.items():
        if extractor is not None:
            if len(df[df.action_name == action_name]) == 0:
                print(label, action_name, "not found in DataFrame")

            # if label:
            #     print(label, action_name)
            # else:
            #     print(action_name)
            # if progress_bar:
            #     df[f"data_{action_name}"] = df[df.action_name == action_name][
            #         "entry_details"
            #     ].progress_apply(extractor)
            # else:
            #     df[f"data_{action_name}"] = df[df.action_name == action_name][
            #         "entry_details"
            #     ].apply(extractor)

    return df


print("ABCD")
df_abcd_conv = match_df_to_actions(df_abcd, progress_bar=True)
# df_abcd_conv.to_parquet(parquet_file_abcd_conv, engine="pyarrow", compression="brotli")
print("MNOP")
df_mnop_conv = match_df_to_actions(df_mnop, progress_bar=True)
# df_mnop_conv.to_parquet(parquet_file_mnop_conv, engine="pyarrow", compression="brotli")
print("ZYXW")
df_zyxw_conv = match_df_to_actions(df_zyxw, progress_bar=True)
# df_zyxw_conv.to_parquet(parquet_file_zyxw_conv, engine="pyarrow", compression="brotli")

ABCD
None SendFuelOrderAction not found in DataFrame
None SetCKIPaxDistributionAction not found in DataFrame
None TdmCreateLoadingInstructionAction not found in DataFrame
MNOP
None SetActualBagWeightIndicatorAction not found in DataFrame
None SetCKIPaxDistributionAction not found in DataFrame
None UpdateTransitLoadTableAction not found in DataFrame
ZYXW
None TransferCheckinDataAction not found in DataFrame
None UpdateEstimatesAction not found in DataFrame


In [ ]:
# print("ABCD")
# df_abcd_conv = extract_df(df_abcd, progress_bar=True)
# df_abcd_conv.to_parquet(parquet_file_abcd_conv, engine="pyarrow", compression="brotli")
# print("MNOP")
# df_mnop_conv = extract_df(df_mnop, progress_bar=True)
# df_mnop_conv.to_parquet(parquet_file_mnop_conv, engine="pyarrow", compression="brotli")
# print("ZYXW")
# df_zyxw_conv = extract_df(df_zyxw, progress_bar=True)
# df_zyxw_conv.to_parquet(parquet_file_zyxw_conv, engine="pyarrow", compression="brotli")

In [8]:
df_zyxw.columns

Index(['id', 'creation_time', 'airline_code', 'flight_number', 'flight_date',
       'departure_airport', 'user_name', 'action_name', 'header_line',
       'entry_details'],
      dtype='object')

In [ ]:
df_zyxw[df_zyxw.flight_date == 13].creation_time.apply(lambda x: str(x)[0:10]).unique()

array(['2024-05-06'], dtype=object)

In [ ]:
print(df_abcd.creation_time.apply(lambda x: str(x)[0:10]).unique())
print(df_mnop.creation_time.apply(lambda x: str(x)[0:10]).unique())
print(df_zyxw.creation_time.apply(lambda x: str(x)[0:10]).unique())

['2024-05-01' '2024-05-02' '2024-05-03' '2024-05-04' '2024-05-06'
 '2024-05-05' '2024-04-30' '2024-05-07']
['2024-04-30' '2024-05-01' '2024-05-02' '2024-05-03' '2024-05-04'
 '2024-05-05' '2024-05-06' '2024-05-07']
['2024-04-30' '2024-05-01' '2024-05-02' '2024-05-03' '2024-05-04'
 '2024-05-05' '2024-05-06' '2024-05-07']


In [15]:
def extract(message: str) -> str | None:
    
    raise NotImplementedError("This message is not supported yet")

In [1]:
# Example on how to effectively filter out different variations of the same message
df = df_mnop
df["entry_details"] = df["entry_details"].apply(remove_typos.remove_typos)

x = df[
    (df.action_name == "SpecialPaxWeightAction")
    # & (~df["entry_details"].isnull())
    # & (
    #     ~df["entry_details"].str.contains("lc2.manualloadplanning.dto.LoadDTO ", na=False)
    # )  # abcd
    # & (~df["entry_details"].str.contains("STATUS AIRCRAFT_CONFIG", na=False))  # abcd
    # & (~df["entry_details"].str.contains("STATUS FUEL", na=False))  # abcd
    # & (
    #     ~df["entry_details"].str.contains("STATUS LOADING_INSTRUCTION", na=False)
    # )  # abcd
    # & (
    #     df["entry_details"].str.contains(
    #         "Message type        :   LOADING_INSTRUCTION", na=False
    #     )
    # )  # abcd
    # & (~df["entry_details"].str.contains("STATUS LOADSHEET", na=False))  # abcd
    # & (~df["entry_details"].str.contains("STATUS LOADING_INSTRUCTION", na=False))  # abcd
    # nMessage type        :   LOADING_INSTRUCTION
    # & (
    #     ~df["entry_details"].str.contains(
    #         "com.systemone.lc2.estimateshandling.dto.EstimateWeightsDTO", na=False
    #     )
    # )  # zyxw
    # & (
    #     ~df["entry_details"].str.contains("STATUS LOADING_INSTRUCTION", na=False)
    # )  # STATUS LOZYING_INSTRUCTION also included
    # & (~df["entry_details"].str.contains("STATUS LOZYING_INSTRUCTION", na=False))
    # & (
    #     ~df["entry_details"].str.contains("Email receivers", na=False)
    # )  # don't include this
    # & (
    #     ~df["entry_details"].str.contains("Telex receivers", na=False)
    # )  # don't include this
]["entry_details"]

NameError: name 'df_mnop' is not defined

In [23]:
i=0

In [32]:

print(x.iloc[i])
i +=1

com.systemone.lc2.estimateshandling.dto.EstimateWeightsDTO [ id = 688840  flightId = 669029 legId = 688840  deleted = false  fragmentId = EstimatesFragment ]

 Pax                 : 207                       Pax Weight          : 16113.0    KG 
 Bag                 : 200                       Bag Weight          : 3000.0     KG 
 Cargo                                                               : 7500.0     KG 
 Mail                                                                : 0.0        KG 
 Tare                : 5                                             : 360.0      KG 
------------------------------------------------------------------------------------------------------------------------
 Traffic Load                                                        : 26973.0    KG 
 DOW                                                                 : 127585.0   KG 
----------------------------------------------------------------------------------------------------------------------

com.systemone.lc2.estimateshandling.dto.EstimateWeightsDTO [ id = 688840  flightId = 669029 legId = 688840  deleted = false  fragmentId = EstimatesFragment ]

 Pax                 : 207                       Pax Weight          : 16113.0    KG 
 Bag                 : 200                       Bag Weight          : 3000.0     KG 
 Cargo                                                               : 7500.0     KG 
 Mail                                                                : 0.0        KG 
 Tare                : 5                                             : 360.0      KG 
------------------------------------------------------------------------------------------------------------------------
 Traffic Load                                                        : 26973.0    KG 
 DOW                                                                 : 127585.0   KG 
------------------------------------------------------------------------------------------------------------------------
 EZFW                                                                : 154558.0   KG
 Remark              :





Pax Weight = 11676.0 KG Bag Weight = 2040.0 KG Cargo = 65.0 KG Mail = 0.0 KG DOW = 53999.0 KG ZFW = 65740.0 KG
STATUS FUEL 2 AIRCRAFT_CONFIG 1 EZFW 2 CARGO_TRANSFER 1 OFP 1 CABIN_CONFIG 1 AUTO_MODE_ACTIVE 1 AUTOMATION_STARTED 0 BAG_LOAD_ITEMS_GEN 1 EZFW_COUNTER 2 REGISTRATION 1 REGISTRATION_CHANGE 3 FUEL_ORDER 1


In [131]:
list(x.unique())

[]

In [75]:
i = 0

In [90]:
print(x.iloc[i])
i += 1

com.systemone.lc2.paxactuals.dto.PaxDataDTO [ id = 9290798 ]
 Baggage weight type: HISTORIC       
CNF            
                Y              Jump           Standby        Male           Female         Child          Infant         Bags           BWgt           Average        
Booked          NULL           NULL           NULL           NULL           NULL           NULL           NULL           0              0.00  KG       NULL           
RAO            
                Y              Jump           Standby        Male           Female         Child          Infant         Bags           BWgt           Average        
Booked          2              NULL           0              2              0              0              0              0              0.00  KG       NULL           

 Distribution   : STANDARD_DISTRIBUTION 
 Section        : 0A             0C             
 Capacity       : Y72            Y102           
 Distribution   : Y0             Y0


In [68]:
list(x.apply(EstimateStorePaxDataAction.extract).unique())

['{"estimated_Y": "1", "estimated_Jump": "NULL", "estimated_Standby": "0", "estimated_Male": "1", "estimated_Female": "0", "estimated_Child": "0", "estimated_Infant": "0", "estimated_Bags": "1", "estimated_BWgt": "12.00", "estimated_Average_BWgt": "12.00"}',
 '{"estimated_Y": "15", "estimated_Jump": "NULL", "estimated_Standby": "0", "estimated_Male": "10", "estimated_Female": "0", "estimated_Child": "5", "estimated_Infant": "4", "estimated_Bags": "0", "estimated_BWgt": "0.00", "estimated_Average_BWgt": "NULL"}',
 '{"estimated_Y": "1", "estimated_Jump": "NULL", "estimated_Standby": "0", "estimated_Male": "0", "estimated_Female": "1", "estimated_Child": "0", "estimated_Infant": "0", "estimated_Bags": "1", "estimated_BWgt": "12.00", "estimated_Average_BWgt": "12.00"}',
 '{"estimated_Y": "22", "estimated_Jump": "NULL", "estimated_Standby": "0", "estimated_Male": "15", "estimated_Female": "1", "estimated_Child": "6", "estimated_Infant": "5", "estimated_Bags": "22", "estimated_BWgt": "264.00

In [60]:
message = "STATUS AIRCRAFT_CONFIG"

In [61]:
if (
    "STATUS AIRCRAFT_CONFIG" in message
    or "STATUS FUEL" in message
    or "STATUS LOADING_INSTRUCTION" in message
):
    print(True)

True


In [ ]:
print(CreateLoadsheetAction.extract(x.iloc[3]))

{"TOTAL TRAFFIC LOAD": 6779, "DRY OPERATING WEIGHT": 44831, "ZERO FUEL WEIGHT ACTUAL": 51610, "TAKE OFF FUEL": 8114, "TAKE OFF WEIGHT ACTUAL": 59724, "TRIP FUEL": 4524, "LANDING WEIGHT ACTUAL": 55200}


In [ ]:
list(x)

['Telex receivers     :   ASRV1ZY\r\nMessage type        :   LOADSHEET\r\nSubject             :   ZY999/30APR GRU-REC; Loadsheet Edition 01\r\nAttachment file name:   ZY999_30APR_GRUREC_Loadsheet_Edition_01\r\nBody                :\r\nZYXW  AIRLINES    OFP:_____  ACCEPTED: _____\r\nL O A D S H E E T              CHECKED            APPROVED     EDNO\r\nALL WEIGHTS IN KILOGRAM                                         01\r\n\r\nFROM/TO FLIGHT      A/C REG     VERSION       CREW     DATE    TIME\r\nGRU REC ZY 999 /30 ZYXRD       Y174          3/4      30APR24 0757\r\n                        WEIGHT             DISTRIBUTION\r\nLOAD IN COMPARTMENTS      5420   1/2210 3/2000 4/1210 5/0\r\nPASSENGER / CABIN BAG    13260  170/0/0          TTL 170\r\n                                Y  170         \r\nTOTAL TRAFFIC LOAD       18680\r\nDRY OPERATING WEIGHT     45204\r\nZERO FUEL WEIGHT ACTUAL  63884  MAX  64300 L  ZYJ\r\nTAKE OFF FUEL             8809\r\nTAKE OFF WEIGHT  ACTUAL  72693  MAX  79000   

DOW 43369  DRY OPERATING WEIGHT
PAY 8805  TOTAL TRAFFIC LOAD
ZFW 52174  MAX   62500    ZERO FUEL WEIGHT ACTUAL
TOF 9674   TAKE OFF FUEL

TOW 61848  MAX   73500   TAKE OFF WEIGHT ACTUAL

TIF 4847   TAXI FUEL
LAW 57001  MAX   66000  L LANDING WEIGHT ACTUAL
UNLD 8999  

In [ ]:
print(extract(x.unique()[0]))

{'TOTAL TRAFFIC LOAD': 12997, 'DRY OPERATING WEIGHT': 44966, 'ZERO FUEL WEIGHT ACTUAL': 57963, 'TAKE OFF FUEL': 5514, 'TAKE OFF WEIGHT ACTUAL': 63477, 'TRIP': 2212, 'LANDING WEIGHT ACTUAL': 61265, 'TAXI FUEL': 286, 'PANTRY': 414}


In [ ]:
y = pd.DataFrame()

In [ ]:
y["entry_details"] = pd.DataFrame(x)

In [ ]:
y["data"] = pd.DataFrame(x.apply(extract))

In [ ]:
y

,entry_details,data
423236,\r\nEmail receivers :\r\nMessage type ...,"{'TOTAL TRAFFIC LOAD': 15200, 'DRY OPERATING W..."
423251,\r\nEmail receivers :\r\nMessage type ...,"{'TOTAL TRAFFIC LOAD': 14975, 'DRY OPERATING W..."
423272,\r\nEmail receivers :\r\nMessage type ...,"{'TOTAL TRAFFIC LOAD': 14701, 'DRY OPERATING W..."
423278,\r\nEmail receivers :\r\nMessage type ...,"{'TOTAL TRAFFIC LOAD': 6779, 'DRY OPERATING WE..."
423303,\r\nEmail receivers :\r\nMessage type ...,"{'TOTAL TRAFFIC LOAD': 18706, 'DRY OPERATING W..."
...,...,...
448385,\r\nEmail receivers :\r\nMessage type ...,"{'TOTAL TRAFFIC LOAD': 14942, 'DRY OPERATING W..."
448396,\r\nEmail receivers :\r\nMessage type ...,"{'TOTAL TRAFFIC LOAD': 15032, 'DRY OPERATING W..."
448411,\r\nEmail receivers :\r\nMessage type ...,"{'TOTAL TRAFFIC LOAD': 8028, 'DRY OPERATING WE..."
448425,\r\nEmail receivers :\r\nMessage type ...,"{'TOTAL TRAFFIC LOAD': 18430, 'DRY OPERATING W..."


In [ ]:
print(extract(x[0]))

{'TOTAL TRAFFIC LOAD': 15200, 'DRY OPERATING WEIGHT': 45523, 'ZERO FUEL WEIGHT ACTUAL': 60723, 'TAKE OFF FUEL': 12502, 'TAKE OFF WEIGHT ACTUAL': 73225, 'TRIP': 4808, 'LANDING WEIGHT ACTUAL': 68417, 'TAXI FUEL': 198, 'PANTRY': 414}
